<a href="https://colab.research.google.com/github/Kausarfatima186/CODSOFT/blob/main/Codsoft_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Sample dataset
data = {
    'User': ['A', 'A', 'A', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'E', 'E'],
    'Movie': ['Matrix', 'Avengers', 'Inception', 'Matrix', 'Titanic', 'Avengers', 'Inception', 'Titanic', 'Matrix', 'Titanic', 'Avengers', 'Inception'],
    'Rating': [5, 4, 3, 5, 4, 4, 5, 3, 3, 4, 5, 4]
}

df = pd.DataFrame(data)

# Create a user-item matrix
user_movie_matrix = df.pivot_table(index='User', columns='Movie', values='Rating').fillna(0)

# Compute user similarity matrix
user_similarity = cosine_similarity(user_movie_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Recommend movies
def recommend_movies(target_user, user_similarity_df, user_movie_matrix, top_n=2):
    # Find similar users
    similar_users = user_similarity_df[target_user].sort_values(ascending=False).drop(target_user).index

    # Calculate weighted ratings for movies the target user hasn't rated
    target_user_ratings = user_movie_matrix.loc[target_user]
    unreviewed_movies = target_user_ratings[target_user_ratings == 0].index
    recommendations = {}

    for movie in unreviewed_movies:
        weighted_sum = 0
        similarity_sum = 0
        for user in similar_users:
            similarity = user_similarity_df.loc[target_user, user]
            rating = user_movie_matrix.loc[user, movie]
            if rating > 0:  # Only consider rated movies
                weighted_sum += similarity * rating
                similarity_sum += similarity
        if similarity_sum > 0:
            recommendations[movie] = weighted_sum / similarity_sum

    # Return the top N recommended movies
    return sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:top_n]

# Example: Recommend movies for User 'A'
recommended_movies = recommend_movies('A', user_similarity_df, user_movie_matrix)
print("Recommended Movies for User 'A':", recommended_movies)


Recommended Movies for User 'A': [('Titanic', 3.611631437761772)]
